In [15]:
import os
import json
import re
import pandas as pd

dodfs = os.listdir('./dodf_json_ate_25_08')

In [16]:
# Regex de atos de aditamento

principal_aditivo = r'(?:ADITIVO)'
regex_titulo_aditivo = r'(?:(ADITIVO[S]*\s.*CONTRAT[OUALIS]*)|(CONTRATO[S]*\s.*ADITIVO[S]*)|(ADITIVO,\s.*CONTRATO))'
regex_texto_aditivo = r'(?:(aditivo\sao\scontrato)|(espécie:\scontrato)|(termo\saditivo\s-\sao\scontrato))'

titulos_termo_aditivo = [
    'EXTRATO DE TERMO ADITIVO',
    'EXTRATO DE ADITIVO',
    'EXTRATO DE TERMO ADITIVO (*)',
    'EXTRATOS DE TERMO ADITIVO',
    'EXTRATOS DE TERMOS ADITIVOS',
    'EXTRATO DO PRIMEIRO TERMO ADITIVO',
    'EXTRATO DE TERMO DE ADITIVO',
    'EXTRATO DE TERMOS ADITIVOS',
]

# Regex de atos de anulação e revogação

regex_anulacao_revogacao = r'(?:AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+REVOGA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+ANULA[CÇ][AÃ]O)'

# Regex de atos de contrato e convênio

principal_contrato = r'(?:CONTRATO)'

regex_titulo_contrato = r'(?:(EXTRATO\sD[OE]\sCONTRATO)|(CONTRATO\sSIMPLIFICADO)|(CONTRATO\sPARA\sAQUISIÇÃO)|(^CONTRATO)'\
                        '|(EXTRATO\sD[OE]\sTERMO\sD[OE]\sCONTRATO\sNº\s)|(EXTRATO\sAO\sCONTRATO)|(CONTRATO\sDE\sPRESTAÇÃO\sDE\sSERVIÇOS\sNº)'\
                        '|(CONTRATO\sDE\sPATROCÍNIO)|(CONTRATO\sDE\sCONCESSÃO\sDE)|(^CONTRATO\sNº)|(^EXTRATOS\sDE\sCONTRATO[S]*))'

principal_convenio = r'(?:(CONVÊNIO)|(CONVENIO))'

regex_titulo_convenio = r'(?:(EXTRATO\sD[OE]\sCONVÊNIO)|(EXTRATO\sDE\sTERMO\sDE\sCONVÊNIO)|(CONVÊNIO\sSIMPLIFICADO)|(CONVÊNIO\sPARA\sAQUISIÇÃO)|(^CONVÊNIO)'\
                        '|(EXTRATO\sD[OE]\sTERMO\sD[OE]\sCONVÊNIO\sNº\s)|(EXTRATO\sAO\sCONVÊNIO)|(CONVÊNIO\sDE\sPRESTAÇÃO\sDE\sSERVIÇOS\sNº)'\
                        '|(CONVÊNIO\sDE\sPATROCÍNIO)|(CONVÊNIO\sDE\sCONCESSÃO\sDE)|(^CONVÊNIO\sNº)|(^EXTRATOS\sDE\sCONVÊNIO[S]*))'

# Regex de atos de licitação

regex_licitacao = r'(?:AVISO\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+PREG[AÃ]O\s+ELETR[OÔ]NICO|AVISOS\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISOS\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISOS\s+D[EO]\s+PREG[AÃ]O\s+ELETR[OÔ]NICO|AVISOS\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITA[CÇ][OÕ]ES|AVISOS?\s+D[EO]\s+LICITA[CÇ][OÕ]ES)'

# Regex de atos e suspensão

regex_suspensao = r'(?:AVISO\s+D[EO]\s+SUSPENS[AÃ]O\s+D[EO]\s+LICITA[CÇ][AÃ]O|AVISO\s+D[EO]\s+SUSPENS[AÃ]O)'

In [17]:
atos_aditamento = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[]
}

atos_anulacao_revogacao = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[]
}

atos_contrato_convenio = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[]
}

atos_licitacao = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[]
}

atos_suspensao = {
    'numero_dodf':[],
    'titulo':[],
    'texto':[]
}

outros = {
    'numero_dodf':[],
    'orgao': [],
    'documento': [],
    'titulo':[],
    'texto':[]
}

In [18]:
for dodf in dodfs:
    if not dodf.endswith('.json'):
        continue

    path = './dodf_json_ate_25_08/' + dodf
    with open(path, 'r') as f:
        file = json.load(f)
        try:
            section_3 = file['json']['INFO']['Seção III']
            for orgao in section_3:
                for documento in section_3[orgao]:
                    for ato in section_3[orgao][documento]:
                        titulo = section_3[orgao][documento][ato]['titulo']
                        texto = section_3[orgao][documento][ato]['texto']
                        
                        if re.search(principal_aditivo, titulo) is not None:
                            atos_aditamento['numero_dodf'].append(file['json']['nu_numero'])
                            atos_aditamento['titulo'].append(titulo)
                            atos_aditamento['texto'].append(re.sub(r'<[^>]*>', '', texto))

                        elif titulo in titulos_termo_aditivo:
                            if re.search(regex_texto_aditivo, texto.lower()) is not None:
                                atos_aditamento['numero_dodf'].append(file['json']['nu_numero'])
                                atos_aditamento['titulo'].append(titulo)
                                atos_aditamento['texto'].append(re.sub(r'<[^>]*>', '', texto))
                        
                        elif re.search(regex_anulacao_revogacao, titulo) is not None:
                            atos_anulacao_revogacao['numero_dodf'].append(file['json']['nu_numero'])
                            atos_anulacao_revogacao['titulo'].append(titulo)
                            atos_anulacao_revogacao['texto'].append(re.sub(r'<[^>]*>', '', texto))
                        
                        elif re.search(principal_contrato, titulo) is not None:
                            if re.search(r'(?:ADITIVO)', titulo) is None:
                                if re.search(regex_titulo_contrato, titulo) is not None:
                                    if 'termo aditivo ao contrato' not in texto.lower():
                                        atos_contrato_convenio['numero_dodf'].append(file['json']['nu_numero'])
                                        atos_contrato_convenio['titulo'].append(titulo)
                                        atos_contrato_convenio['texto'].append(re.sub(r'<[^>]*>', '', texto))
                        
                        elif re.search(principal_convenio, titulo) is not None:
                            if re.search(r'(?:ADITIVO)', titulo) is None:
                                if re.search(regex_titulo_convenio, titulo) is not None:
                                    if 'termo aditivo ao con' not in texto.lower():
                                        atos_contrato_convenio['numero_dodf'].append(file['json']['nu_numero'])
                                        atos_contrato_convenio['titulo'].append(titulo)
                                        atos_contrato_convenio['texto'].append(re.sub(r'<[^>]*>', '', texto))

                        elif re.search(regex_licitacao, titulo) is not None:
                            atos_licitacao['numero_dodf'].append(file['json']['nu_numero'])
                            atos_licitacao['titulo'].append(titulo)
                            atos_licitacao['texto'].append(re.sub(r'<[^>]*>', '', texto))

                        elif re.search(regex_suspensao, titulo) is not None:
                            atos_suspensao['numero_dodf'].append(file['json']['nu_numero'])
                            atos_suspensao['titulo'].append(titulo)
                            atos_suspensao['texto'].append(re.sub(r'<[^>]*>', '', texto))

                        else:
                            outros['numero_dodf'].append(file['json']['nu_numero'])
                            outros['orgao'].append(orgao)
                            outros['documento'].append(ato)
                            outros['titulo'].append(titulo)
                            outros['texto'].append(re.sub(r'<[^>]*>', '', texto))

        except KeyError:
            print(f"Chave 'Seção III' não encontrada no DODF {file['lstJornalDia']}!")


Chave 'Seção III' não encontrada no DODF ['DODF 060 09-07-2022 EDICAO EXTRA A.pdf\n']!
Chave 'Seção III' não encontrada no DODF ['DODF 076 28-08-2021 EDICAO EXTRA A.pdf\n']!
Chave 'Seção III' não encontrada no DODF ['DODF 076 28-08-2021 EDICAO EXTRA A.pdf\n']!


In [19]:
df_atos_aditamento = pd.DataFrame(atos_aditamento)
df_atos_anulacao_revogacao = pd.DataFrame(atos_anulacao_revogacao)
df_atos_contrato_convenio = pd.DataFrame(atos_contrato_convenio)
df_atos_licitacao = pd.DataFrame(atos_licitacao)
df_atos_suspensao = pd.DataFrame(atos_suspensao)
df_outros = pd.DataFrame(outros)

In [20]:
df_atos_aditamento

,numero_dodf,titulo,texto
0,154,EXTRATO DO TERMO ADITIVO AO CONTRATO N° 02/2022,Processo nº 00134-00000513/2022-73. Contratant...
1,154,II TERMO ADITIVO AO CONTRATO BRB Nº 143/2020,Contratada: Ana Paula da Silva Di Pace Braz. O...
2,154,II TERMO ADITIVO AO CONTRATO BRB Nº 151/2020,Contratada: Constru - Eng Engenharia de Avalia...
3,154,II TERMO ADITIVO AO CONTRATO BRB Nº 152/2020,Contratada: Diony Gomes Porto. Objeto contrato...
4,154,II TERMO ADITIVO AO CONTRATO BRB Nº 153/2020,"Contratada: D-BANK Engenharia, Avaliação e Per..."
...,...,...,...
3387,107,EXTRATO DO PRIMEIRO TERMO ADITIVO AO TERMO DE ...,Processo: 00090-00002058/2022-86. Das Partes: ...
3388,107,TERMO ADITIVO DE ATA DE REGISTRO DE PREÇOS,1° TERMO ADITIVO À ATA DE REGISTRO DE PREÇOS N...
3389,107,EXTRATO DE ADITIVO,Primeiro Termo Aditivo ao Contrato nº 09/2022-...
3390,107,EXTRATO DE TERMO ADITIVO,Processo: 0196-000171/2016. Da Espécie: Tercei...


In [21]:
df_atos_anulacao_revogacao

,numero_dodf,titulo,texto
0,82,AVISO DE REVOGAÇÃO PREGÃO ELETRÔNICO Nº 10/2021,Processo: 00080-00073562/2020-36.O Subsecretár...
1,145,AVISO DE REVOGAÇÃO DE LICITAÇÃO,O Presidente da Comissão de Licitação para Com...
2,142,AVISO DE REVOGAÇÃO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 07/2021(AMPLA CONCORRÊNCI...
3,167,AVISO DE REVOGAÇÃO,PREGÃO ELETRÔNICO Nº 81/2021Objeto: A presente...
4,221,AVISO DE REVOGAÇÃO,CONCORRÊNCIA Nº 10/2013Processo:113.0013235/20...
...,...,...,...
70,83,AVISO DE REVOGAÇÃO,PREGÃO ELETRÔNICO Nº 16/2021Processo: 04026-00...
71,41,AVISO DE ANULAÇÃO DE LICITAÇÃO,CONCORRÊNCIA N° 03/2022O Jardim Botânico de Br...
72,41,AVISO DE ANULAÇÃO DE LICITAÇÃO,CONCORRÊNCIA N° 04/2022O Jardim Botânico de Br...
73,41,AVISO DE ANULAÇÃO DE LICITAÇÃO,CONCORRÊNCIA N° 05/2022O Jardim Botânico de Br...


In [22]:
df_atos_contrato_convenio

,numero_dodf,titulo,texto
0,154,EXTRATO DO CONTRATO Nº 047.029 - RA - XXXI,Processo: 00367-00000214/2022-50. Contrato: nº...
1,154,EXTRATO DO CONTRATO Nº 47031/2022 - SEEC,Processo nº 00040-00026965/2022-05. A SECRETAR...
2,154,EXTRATO DO CONTRATO DE PRESTAÇÃO DE SERVIÇOS N...,PROCESSO SEI-GDF Nº 00413-00002160/2022-18. PA...
3,154,EXTRATO DO CONTRATO BRB Nº 294/2022,Contratada: PENTA SIS AUTOMAÇÃO. Modalidade: P...
4,154,EXTRATO DO TERMO DO CONTRATO Nº 54/2022,PROCESSO nº 00113-00020545/2021-15; CONTRATANT...
...,...,...,...
3213,107,EXTRATO DO CONTRATO Nº 18/2022,Processo: 00054-00006759/2021-40. PARTES: DF/P...
3214,107,CONTRATO DE PRESTAÇÃO DE SERVIÇOS Nº 19/2022 -...,"O Distrito Federal, por meio do Departamento d..."
3215,107,EXTRATO DE CONTRATO,CONTRATO Nº 9477. ASSINATURA: 06/06/2022. Proc...
3216,107,EXTRATO DE CONTRATO,Espécie: Contrato nº 05/2022-CEB Geração S.A. ...


In [23]:
df_atos_licitacao

,numero_dodf,titulo,texto
0,154,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO SRP Nº 111/2022 - UASG 97400...
1,154,AVISO DE LICITAÇÃO,TOMADA DE PREÇOS Nº 01/2022 - UASG 974002A Com...
2,154,AVISO DE ABERTURA DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 13/2022 - UASG 926334A Fu...
3,154,AVISO DE LICITAÇÃO - REABERTURA (*),PREGÃO ELETRÔNICO Nº 04/2022Processo SEI-GDF n...
4,154,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 07/2022Processo nº 00054-...
...,...,...,...
1950,107,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 15/2022Processo: 00054-00...
1951,107,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 65/2022A presente licitaç...
1952,107,AVISO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 66/2022A presente licitaç...
1953,107,AVISO DE LICITAÇÃO (*),PREGÃO ELETRÔNICO Nº 117/2022Processo: 092-000...


In [24]:
df_atos_suspensao

,numero_dodf,titulo,texto
0,8,AVISO DE SUSPENSÃO,PREGÃO ELETRÔNICO POR SRP Nº 444/2021 - UASG 9...
1,8,AVISO DE SUSPENSÃO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 37/2021Processo: 00010.00...
2,197,AVISO DE SUSPENSÃO DE LICITAÇÃO,PREGÃO ELETRÔNICO Nº 25/2021Processo: 00050-00...
3,82,AVISO DE SUSPENSÃO,PREGÃO ELETRÔNICO Nº 03/2022 - SRPProcesso: 00...
4,120,AVISO DE SUSPENSÃO,CONCORRÊNCIA Nº 07/2022A presente licitação qu...
...,...,...,...
190,176,AVISO DE SUSPENSÃO,PREGÃO ELETRÔNICO POR SRP Nº 339/2021 - UASG 9...
191,176,AVISO DE SUSPENSÃO,PREGÃO ELETRÔNICO Nº 17/2021Processo: 00054-00...
192,176,AVISO DE SUSPENSÃO,PREGÃO ELETRÔNICO N° 15/2021 (UASG: 927507)Pro...
193,39,AVISO DE SUSPENSÃO,PREGÃO ELETRÔNICO Nº 01/2022Processo: 00094-00...


In [25]:
df_outros

,numero_dodf,orgao,documento,titulo,texto
0,154,Poder Legislativo,235073,EXTRATO DE INEXIGIBILIDADE DE LICITAÇÃO,Processo: 00001-00025785/2022-19. Favorecido: ...
1,154,Poder Legislativo,235301,AVISO DE JULGAMENTO,PREGÃO ELETRÔNICO Nº 27/2022Processo nº 00001-...
2,154,Secretaria de Estado de Governo,235271,EDITAL Nº 03/2022,RETIFICAÇÃORETIFICA O EDITAL DE CONCORRÊNCIA P...
3,154,Secretaria de Estado de Governo,235090,RATIFICAÇÃO DE DISPENSA DE LICITAÇÃO,PROCESSO: 00367-00000214/2022-50. INTERESSADO:...
4,154,Secretaria de Estado de Economia,235081,EXTRATO DO TERMO DE ADESÃO Nº 42/2022 - SEEC,Processo nº 00040-00030031/2022-60 – A SECRETA...
...,...,...,...,...,...
29471,107,Secretaria de Estado de Trabalho,217967,AVISO DE CONVOCAÇÃO,EDITAL DE CHAMAMENTO PÚBLICO Nº 17/2022O Gover...
29472,107,Ineditorial,217961,CHAMAMENTO Nº 226/2022,PROCESSO: 04024-00005423/2022-31O Instituto do...
29473,107,Ineditorial,217292,AVISO DE REQUERIMENTO DE RENOVAÇÃO DE LICENÇA ...,CNPJ: 09.232.995/0001-90Torna público que está...
29474,107,Ineditorial,217643,EDITAL DE CONVOCAÇÃO,A Diretoria do Sindicato dos Servidores Públic...
